In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
#import lazypredict

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from sklearn.ensemble import StackingRegressor




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Первичный осмотр данных

In [ ]:
# загружаем тренеровочный и тестовый датасеты
df_test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
df_train = pd.read_csv('../input/parsed-auto/final_auto.csv')
df_sub = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

In [ ]:
# осмотрим тестовые и тренеровочные данные:
for i in [df_train, df_test]:
    print(i.info())

In [ ]:
# удалим столбцы, которых нет в нашем тренеровочном датасете
df_test.drop(['complectation_dict', 'car_url', 'Привод', 'Руль', 'Состояние', 'Таможня','parsing_unixtime',
              'model_info', 'mileage', 'vendor', 'Владельцы', 'image', 'equipment_dict',
              'priceCurrency', 'super_gen', 'Владение', 'ПТС', 'name'
             ], axis=1, inplace=True)



In [ ]:
# добавим в тестовый датафрейм столбец с итоговым значением для корректной обработки данных в дальнейшем
df_test['price'] = 0

# отметим тренеровочный и тестовый
df_test['markup'] = 1
df_train['markup'] = 0

# удалим в train столбец, дублирующий индексы
df_train.drop(['Unnamed: 0'], axis=1, inplace=True)

# соединим их вместе

data = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем

In [ ]:
# осмотрим рабочие данные
data.info()

****Немног о данных:****

1. bodyType - тип кузова
2. brand - марка автомобиля
3. color - цвет автомобиля
4. description - описание 
5. engineDisplacement - рабочий объём двигателя
6. enginePower - мощность двигателя
7. fuelType - тип топлива
8. modelDate - год модели
9. model_name - занвание модели
10. numberOfDoors - количество дверей
11. productionDate - год выпуска авто
12. vehicleConfiguration - конфигурация авто
13. vehicleTransmission - тип трансмиссии

In [ ]:
# проверим на дубликаты

len(data) - len(data.drop_duplicates())

дубликаты не несут никакой информации. необходимо удалить их

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
# проверим на наличие пропусков

data.isna().sum()

пропуски присутствую только в столбце sell_id в тестировочных данных, так как по sell_id
будут отправляться на сабмит. 

In [ ]:
# удалим пропуски, чтобы в дальнейшем не обращать на это внимание 
data.fillna(0, inplace=True)

# EDA

Для удобства, приведём названия признаком в одному типу

In [ ]:
data.rename(columns= {
    'bodyType': 'body_type',                              
    'engineDisplacement': 'engine_displacement',       
    'enginePower': 'engine_power',                
    'fuelType': 'fuel_type',                  
    'modelDate': 'model_date',                    
    'numberOfDoors': 'numbers_of_doors',               
    'productionDate':  'production_date',
    'vehicleConfiguration': 'vehicle_configuration', 
    'vehicleTransmission': 'vehicle_transmission'        
    }, inplace=True)

Проведём первичный EDA с помощью pandas_profiling

In [ ]:
#pandas_profiling.ProfileReport(data)

**Обработка признаков**

**1. body_type**

In [ ]:
data.body_type.value_counts()

Некорректные данные признака отсутствуют. Данные необходимо унифицировать

In [ ]:
# Создадим словарь для унификации:
body_type_dict = {
    'внедорожник 5 дв.': 'outlander 5 d',      
    'седан': 'sedan',                     
    'лифтбек': 'liftback',                   
    'хэтчбек 5 дв.': 'hatchback 5d',              
    'универсал 5 дв.': 'station_wagon',            
    'минивэн': 'minivan',                    
    'купе': 'coupe',                        
    'компактвэн': 'compactvan',                 
    'хэтчбек 3 дв.': 'hatchback 3d',               
    'пикап двойная кабина': 'pickup 2 cab',        
    'внедорожник 3 дв.': 'outlander 3d',           
    'купе-хардтоп': 'coupe-hardtop',                
    'кабриолет': 'cabriolet',                  
    'фургон': 'van',                       
    'родстер': 'rodster',                     
    'микровэн': 'microvan',                   
    'седан-хардтоп': 'senad-hardtop',                
    'пикап одинарная кабина': 'pickup 1 cab',        
    'пикап полуторная кабина': 'pickup 1.5 cab',       
    'лимузин': 'limousine',                       
    'седан 2 дв.': 'sedan 2d',                   
    'внедорожник открытый': 'outlander open',           
    'тарга': 'targa',                          
    'фастбек': 'fastback'               
}

# заменим данные столбца на унифицированные данные:
data['body_type'] = data['body_type'].map(body_type_dict)

In [ ]:
sns.countplot(y = data['body_type'], data = data)

Самые пополярные типы кузовов - седан и 5-дверный внедорожник

**2. color**

In [ ]:
data['color'].value_counts()

In [ ]:
# создадим словарь для унификации и заменим на них данные признака
color_dict = {
    'чёрный': 'black',         
    'белый': 'white',          
    'серый': 'gray',           
    'серебристый': 'light gray',     
    'синий': 'dark',           
    'коричневый': 'brown',      
    'красный': 'red',        
    'зелёный': 'green',         
    'бежевый': 'beige',         
    'голубой': 'light_blue',          
    'золотистый': 'golded',      
    'пурпурный': 'purple',      
    'фиолетовый': 'violet',       
    'жёлтый': 'yellow',           
    'оранжевый': 'orange',       
    'розовый': 'pink'
}

data['color'] = data['color'].map(color_dict)

In [ ]:
sns.countplot(y = data['color'], data = data)

Самые популярные цвета - черный и белый

**3. brand**

In [ ]:
data['brand'].value_counts()

In [ ]:
sns.countplot(y = data['brand'], data = data)

Данные чистые. Infiniiti и Lexus представленые мешьне всего. Можно предположить, что это из-за того, что это авто премиум класса и распространеные не так сильно, как их конкуренты

**4. description**         

In [ ]:
data['description']

Признак с описанием авто от его владельца. Вытащить оттуда закономерности крайне сложная задача. В этом кейсе использовать его не будем

In [ ]:
# удаляем

data.drop(['description'], axis=1, inplace=True)

**5. engine_displacement**         

In [ ]:
data['engine_displacement'].value_counts()

у 151 автомобиля не указан рабочий объём двигателя. посмотрим, с чем это может быть связано

In [ ]:
data[data['engine_displacement']==' LTR'][['model_name', 'brand', 'fuel_type']].value_counts()

не указан у электрокаров, что логично

In [ ]:
# очистим признак от ненужных букв и заменим пустые значения в электрокарах на 0

data['engine_displacement'] = data['engine_displacement'].apply(lambda x: 0 if x==' LTR' else x)
data['engine_displacement'] = data['engine_displacement'].astype(str).apply(lambda x: x.split()[0])
data['engine_displacement'] = data['engine_displacement'].astype(float)

In [ ]:
data['engine_displacement']

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(y = data['engine_displacement'], data = data)

Самые часто встречающеимся автомобили - средлелитражные. с объемомо двигателя 2.0, 1.6 и 3.0

**6. engine_power**

In [ ]:
data['engine_power'].unique()

In [ ]:
# очистим он ненужных символов
data['engine_power'] = data['engine_power'].str.replace(' N12', '')
data['engine_power'] = data['engine_power'].astype(int)

In [ ]:
data['engine_power'].describe()

Максимальная мощность двигателя - 646л.с., минимальная  30 л.с., средняя - 191 л.с.

Так как мощность двигателя - дискретная величина, для дальнейшей обработки, разобьем на категории

In [ ]:
def engine_power_func(x):
    """ функция для разделения признака 'engine_power' на категории"""
    if x < 100: x = 1
    elif x < 150: x = 2
    elif x < 200: x = 3
    elif x < 250: x = 4
    elif x < 300: x = 5
    elif x < 350: x = 6
    elif x < 400: x = 7
    elif x < 450: x = 8
    elif x < 500: x = 9
    elif x < 550: x = 10
    elif x < 600: x = 11
    else: x = 12
    return x  

In [ ]:
data['engine_power'] = data['engine_power'].apply(engine_power_func)

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(y = data['engine_power'], data = data)

Самые распространённые автомобили с мощностью двигателя от 100 до 200 лошадиных сил

**7. fuel_type**       

In [ ]:
data['fuel_type'].value_counts()

In [ ]:
# создадим словарь для унификации и заменим на них данные признака

fuel_dict = {
    'бензин': 'petrol',
    'дизель': 'dt',
    'гибрид': 'hybrid',
    'электро': 'electro',
    'газ': 'gas'
}

data['fuel_type'] = data['fuel_type'].map(fuel_dict)

In [ ]:
sns.countplot(data=data, y=data['fuel_type'])

Самай распространённый тип топлива в автомобилях - бензин

**8. model_date**

In [ ]:
data['model_date'].value_counts().sort_index()

In [ ]:
data['model_date'].value_counts().sort_values()

Отсортировав по индексу и по количетству значений индекса, можно прийти к выводу, что данные чистые. Чаще всего встречаются довольно "свежие" модели автомобилей.

**9. model_name**

In [ ]:
data['model_name'].value_counts()

Чаще всего встречаются Шкода Октавиа

**10. number_of_doors**

In [ ]:
data['numbers_of_doors'].value_counts()

у одного автомобиля указано 0 дверей. взглянем на него

In [ ]:
data[data['numbers_of_doors'] == 0]

у этого автомобиля действительно 0 дверей

**11. production_date**

In [ ]:
data['production_date'].value_counts()

In [ ]:
data['production_date'].value_counts().sort_index()

отсортировав значения по индексам, можно прийти к выводу, что данные чистые

**12. vehicle_сonfiguration**

In [ ]:
data['vehicle_configuration'].value_counts()

признак дублирует обобщенную информацию из других признаков. обработать крайне сложно. удалим его

In [ ]:
data.drop(['vehicle_configuration'], axis=1, inplace=True)

**13. vehicle_transmission**         

In [ ]:
data['vehicle_transmission'].value_counts()

In [ ]:
# создадим словарь для унификации и заменим на них данные признака

transmission_dict = {
  'автоматическая': 'auto',
    'механическая': 'mech',
    'вариатор': 'var',
    'роботизированная': 'robo'
}

data['vehicle_transmission'] = data['vehicle_transmission'].map(transmission_dict)

In [ ]:
sns.countplot(data=data, y='vehicle_transmission')

In [ ]:
# создадим признак years_in_use, на основании production_date

data['years_in_use'] = 2021 - data['production_date'] 

#

In [ ]:
data['production_date'].value_counts()

самая распространённая трансмиссия - автоматическая

# **Анализ признаков**

In [ ]:
# выделим используемые признаки
used_features = ['body_type', 'brand', 'color', 'engine_displacement', 'engine_power',
       'fuel_type', 'model_date', 'model_name', 'numbers_of_doors',
       'production_date', 'vehicle_transmission', 'years_in_use']

In [ ]:
# преобразуем признаки в понятные для машины данные
le = LabelEncoder()
for i in used_features:
    le.fit(data[i])
    data[i] = le.transform(data[i])

In [ ]:
# посмотрим на значимость признаков 
imp_cat = pd.Series(mutual_info_classif(data[data['price']>0][used_features],
                                     data[data['price']>0]['price'],
                                     discrete_features = True), index=used_features)
#plt.figure(figsize=(15,15))
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

# Построение моделей

In [ ]:
def mape(y_test, y_pred):
    """функция для основной метрики"""
    return np.mean(np.abs((y_test - y_pred) / y_test))

**1. Линейная регрессия**

In [ ]:
X, y = data[data['markup']==0][used_features], data[data['markup']==0]['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAPE = {mape(y_test, y_pred)}%')

**2. Градиентный бустинг**

In [ ]:
# зададим сетку параметров
param_grid = {
    'n_estimators': [500, 1000],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2,3],  
}
# найдем лучшие параметры модели
#model = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
#model.fit(X_train, y_train)
#print(f'лучшие параметры модели - {model.best_params_}')

Получаем лучшие параметры модели
лучшие параметры модели - {'max_depth': 5, 'min_samples_split': 3, 'n_estimators': 1000}

In [ ]:
model = GradientBoostingRegressor(random_state=0, max_depth=5, min_samples_split=3, n_estimators=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAPE = {mape(y_test, y_pred)*100}%')

**3. CatBoost**

In [ ]:
CBR = CatBoostRegressor(iterations = 5000,
                       random_state = 0,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'])

CBR.fit(X_train, y_train,
             eval_set=(X_test, y_test),
             verbose_eval=0,
             use_best_model=True)

y_pred = CBR.predict(X_test)

print(f'MAPE = {mape(y_test, y_pred)*100}%')

**4. XGBoostRegressor**

In [ ]:
# зададим сетку параметров
param_grid = {
    'n_estimators': [500, 1000, 2000],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2,3]   # 5, 10
}

#model = GridSearchCV(estimator=XGBRegressor(random_state=42), param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
#model.fit(X_train, y_train)
#print(f'лучшие параметры модели - {model.best_params_}')

In [ ]:
xgbr = XGBRegressor(random_state=0, max_depth=5, min_samples_split=2, n_estimators=500)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)

print(f'MAPE = {mape(y_test, y_pred)*100}%')

**5. Стэкинг**

За основу final_estimator возьмем алгоритм, предсказавший лучший результат

In [ ]:

estimators = [('xgb', XGBRegressor(random_state=0, 
                                   max_depth=5, 
                                   min_samples_split=2, 
                                   n_estimators=500)),
              ('cbr', CatBoostRegressor(iterations = 5000,
                       random_state = 0,
                       eval_metric='MAPE',
                        silent=True)),
]
final_estimator = GradientBoostingRegressor(random_state=0, 
                                            max_depth=5, 
                                            min_samples_split=3, 
                                            n_estimators=1000)
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=final_estimator)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f'MAPE = {mape(y_test, y_pred)*100}%')

# Итог

1. Лучшей моделью, показавший метрику MAPE = 12.7%, оказался Градиентный бустинг. 
2. Сильно на модель могли повлиять выбросы, так как присутствуют старые модели автомобилей, цену на которые очень сложно предсказать. Так как они присустствуют в малом количеcтве

In [ ]:
###

In [ ]:
test_price = model.predict(data[data['markup']==1].drop(['sell_id', 'price', 'markup'], axis=1))

In [ ]:
df_sub['price'] = test_price

In [ ]:
df_sub.to_csv('submission_0.csv', index=False)